### 构建一个语义搜索引擎

本教程将帮助你熟悉 LangChain 的几个核心组件：**文档加载器（document loader）**、**嵌入模型（embedding）** 和 **向量存储（vector store）** 的抽象接口。这些抽象的设计目的是为了支持从（向量）数据库和其他数据源中检索数据，以便将其集成到大型语言模型（LLM）的工作流程中。

这些功能对于那些在模型推理过程中需要结合外部数据进行思考的应用非常重要，例如在 **检索增强生成（Retrieval-Augmented Generation，RAG）** 中的应用。

在本教程中，我们将构建一个针对 PDF 文档的搜索引擎。这将使我们能够根据输入的查询语句，检索出 PDF 中与之相似的段落内容。

---

### 涉及的核心概念

本指南主要介绍文本数据的检索方法，涵盖以下核心概念：

1. **文档与文档加载器（Documents and document loaders）**  
   如何加载和表示文档数据。

2. **文本分割器（Text splitters）**  
   将大段文本切分为适合处理的小块内容。

3. **嵌入模型（Embeddings）**  
   将文本转换为可用于语义匹配的向量表示。

4. **向量存储与检索器（Vector stores and retrievers）**  
   存储和高效检索向量化后的文本片段。

### 安装

本教程需要安装 `langchain-community` 和 `pypdf` 两个 Python 包：

In [1]:
%pip install --upgrade --quiet langchain-community pypdf

Note: you may need to restart the kernel to use updated packages.


### 文档和文档加载器
LangChain 实现了一个 **Document（文档）** 抽象类，用于表示一段文本内容及其相关的元数据。它具有三个属性：

- `page_content`：一个字符串，表示文档的内容；
- `metadata`：一个字典，包含任意的元数据；
- `id`：（可选）文档的唯一标识符。

其中，`metadata` 属性可以记录文档来源、与其他文档的关系等信息。请注意，一个单独的 `Document` 对象通常代表一个更大文档中的“块”（chunk）。  

LangChain 生态系统中实现了许多**文档加载器（document loaders）**，它们与数百种常见的数据源进行了集成。这使得将这些数据源中的内容整合到你的 AI 应用程序中变得非常简便。

在需要时，我们也可以生成示例文档：

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="狗是很好的伙伴，以忠诚和友好而闻名。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="猫是独立的宠物，通常喜欢拥有自己的空间。",
        metadata={"source": "mammal-pets-doc"},
    ),
]

### 加载文档

让我们将一个 PDF 文件加载为一系列 `Document` 对象。

在 LangChain 仓库中有一个示例 PDF 文件 —— 这是耐克公司（Nike）2023 年的 10-K 财报文件。

我们可以参考 [LangChain 文档](https://python.langchain.com/docs/integrations/document_loaders/#pdfs) 中关于支持的 PDF 文档加载器的说明。我们选择使用 `PyPDFLoader`，这是一个相对较轻量级的加载器。

然后你就可以使用 `PyPDFLoader` 来加载 PDF 文件了。


In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../assets/example_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


`PyPDFLoader` 会为 PDF 中的每一页加载一个 `Document` 对象。对于每一个对象，我们可以轻松访问：

- 页面的字符串内容（`page_content`）；
- 包含文件名和页码的元数据（`metadata`）。

In [4]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../assets/example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


### 分割（Splitting）

出于信息检索和下游问答的目的，单个 PDF 页面的粒度可能过于粗糙。我们最终的目标是检索能够回答输入查询的 `Document` 对象，因此进一步对文档进行分割将有助于确保文档中相关部分的含义不会被周围文本“淹没”。

我们可以使用 **文本分割器（text splitters）** 来实现这一目标。在这里，我们将使用一个基于字符的简单文本分割器。我们将文档分割为每块 1000 个字符，并在块之间保留 200 个字符的重叠部分。这种重叠有助于避免将某个语句与其相关的上下文分隔开。

我们使用的是 `RecursiveCharacterTextSplitter`，它会递归地使用常见的分隔符（如换行符）对文档进行分割，直到每个块达到合适的大小。这是推荐用于通用文本场景的文本分割器。

我们设置 `add_start_index=True`，这样每个分割后的 `Document` 在原始文档中开始的字符索引会被保存为元数据中的 `"start_index"` 属性。

---

📌 **想了解更多关于如何处理 PDF 的内容**，包括如何提取特定章节的文本和图像，请参阅 [LangChain 关于 PDF 处理的指南](https://python.langchain.com/docs/how_to/document_loader_pdf/)。

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

In [6]:
all_splits[0]

Document(metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../assets/example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1', 'start_index': 0}, page_content="Table of Contents\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE FISCAL YEAR ENDED MAY 31, 2023\nOR\n☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE TRANSITION PERIOD FROM                         TO                         .\nCommission Fil

### 词嵌入
**向量搜索** 是一种用于存储和检索非结构化数据（例如非结构化文本）的常用方法。其核心思想是将与文本相关的数值向量进行存储。当给定一个查询时，我们可以将其也嵌入为相同维度的向量，并使用向量相似度度量方法（如余弦相似度）来识别相关的文本内容。

LangChain 支持来自多个提供商的嵌入模型（embeddings），这些模型定义了如何将文本转换为数值向量。下面我们选择一个模型：

### ✅ 正确搭配建议

| 场景 | 推荐 Embedding 模型 | 推荐 LLM |
|------|---------------------|-----------|
| 文本 Embedding + 语义检索 | `text-embedding-v1/v2` | N/A |
| 多模态理解（图文） | N/A | `qwen-vl-max` |
| RAG（基于文本检索 + 生成） | `text-embedding-v1/v2` | `qwen-turbo` / `qwen-plus` / `qwen-max` |

---

### ✅ 示例：正确搭配使用

```python
# 使用 Qwen 的 Embedding 模型进行向量化
from langchain_community.embeddings import TongyiEmbeddings
embeddings = TongyiEmbeddings(model="text-embedding-v1")

# 使用 Qwen 的 LLM 进行回答生成
from langchain_community.chat_models import ChatTongyi
chat_model = ChatTongyi(model="qwen-turbo")
```

---

### 📌 总结

| 模型名称 | 类型 | 是否可用于 Embedding | 是否适合与 qwen-vl-max 搭配 |
|----------|------|------------------------|------------------------------|
| `qwen-vl-max` | 多模态 LLM | ❌（不适合） | ❌ 不推荐作为 Embedding 使用 |
| `text-embedding-v1/v2` | Embedding 模型 | ✅ | ✅ 推荐用于语义检索等任务 |

In [11]:
# %pip install --upgrade --quiet  langchain-community dashvector dashscope

In [9]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    _ = load_dotenv()
except ImportError:
    pass

if not os.environ.get("DASHSCOPE_API_KEY"):
  os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_community.embeddings.dashscope import DashScopeEmbeddings
# 初始化 Qwen Embedding 模型
embeddings = DashScopeEmbeddings(model="text-embedding-v1")  

In [10]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1536

[-3.75884747505188, -13.396482467651367, 7.487827777862549, 4.250734806060791, 2.5559475421905518, 2.882380485534668, -5.610165596008301, -0.058700062334537506, 0.01584913395345211, -0.28392794728279114]


在拥有了用于生成文本嵌入（text embeddings）的模型之后，接下来我们可以将这些向量存储在一个特殊的数据结构中，该结构支持高效的相似性搜索。

### 向量存储（Vector Stores）

LangChain 中的 `VectorStore` 对象包含用于**添加文本和文档对象**到存储中，以及使用各种**相似性度量方法**进行查询的方法。它们通常在初始化时会绑定一个嵌入模型（embedding model），该模型决定了文本数据如何被转换为数值向量。

LangChain 提供了一系列与不同向量存储技术集成的功能：

- 有些向量存储是由服务提供商托管的（例如各类云服务商），使用时需要特定的凭证；
- 有些（如 Postgres）可以在本地运行，或通过第三方服务部署；
- 还有一些可以在内存中运行，适用于轻量级任务。

In [12]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

实例化我们的向量存储后，我们现在可以索引文档。

In [13]:
ids = vector_store.add_documents(documents=all_splits)

In [22]:
ids[:5]

['9a432618-ff75-4d61-9075-050817d902ec',
 '24f25b57-8265-4ddf-affc-036e55ed473d',
 '4adccd4c-db5c-40f3-9270-04717288ffb4',
 'ecfcea94-ae1e-427e-bb1f-f29f134a4f84',
 '8c0bb7b9-2ada-4b41-b2c7-2d1465b58dd8']

请注意，大多数向量存储（Vector Store）的实现都允许你连接到一个**已有的向量存储**——例如，通过提供客户端对象、索引名称或其他相关信息。具体操作可以参考对应集成的文档以获得更详细的说明。

一旦我们实例化了一个包含文档的 `VectorStore`，就可以对其进行查询。`VectorStore` 提供了多种查询方式：

- **同步查询 和 异步查询**；
- **通过字符串查询 和 通过向量查询**；
- **返回相似度分数 和 不返回相似度分数**；
- **按相似性排序查询 和 按最大边缘相关性（MMR）查询**（用于在结果中平衡与查询的相关性和结果之间的多样性）。

这些查询方法的返回结果通常都包含一个 `Document` 对象的列表。

### 使用方式

嵌入模型（Embeddings）通常将文本表示为一个“稠密”向量，使得在语义上相近的文本在几何空间中距离较近。这使我们能够仅通过传入一个问题来检索相关信息，而无需事先知道文档中使用的任何特定关键词。

根据与字符串查询的相似性返回文档：

In [14]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 

**异步查询：**

指的是在不阻塞主线程的情况下执行的查询操作。在处理大型语言模型或远程向量数据库时，查询可能会耗时较长，使用异步方式可以提高程序的响应性和效率。

在 LangChain 中，某些 `VectorStore` 实现支持异步方法（如 `asimilarity_search()`），你可以在异步函数中调用它们，通常与 `asyncio` 一起使用。

In [15]:
results = await vector_store.asimilarity_search("When was Nike incorporated?") # 耐克是什么时候成立的
 
print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'cr

**返回分数：**

In [19]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")  # 耐克 2023 年的收入是多少？
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.7932733865566657

page_content='Enterprise Resource Planning Platform, data and analytics, demand sensing, insight gathering, and other areas to create an end-to-end technology foundation, which we
believe will further accelerate our digital transformation. We believe this unified approach will accelerate growth and unlock more efficiency for our business, while driving
speed and responsiveness as we serve consumers globally.
FINANCIAL HIGHLIGHTS
• In fiscal 2023, NIKE, Inc. achieved record Revenues of $51.2 billion, which increased 10% and 16% on a reported and currency-neutral basis, respectively
• NIKE Direct revenues grew 14% from $18.7 billion in fiscal 2022 to $21.3 billion in fiscal 2023, and represented approximately 44% of total NIKE Brand revenues for
fiscal 2023
• Gross margin for the fiscal year decreased 250 basis points to 43.5% primarily driven by higher product costs, higher markdowns and unfavorable changes in foreign
currency exchange rates, partially offset 

根据嵌入向量的相似性返回文档：

指通过将查询文本转换为向量（即嵌入），然后在向量数据库中查找与该向量最相似的文档。这种方式不依赖关键词匹配，而是基于语义相似性进行检索。

In [17]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Enterprise Resource Planning Platform, data and analytics, demand sensing, insight gathering, and other areas to create an end-to-end technology foundation, which we
believe will further accelerate our digital transformation. We believe this unified approach will accelerate growth and unlock more efficiency for our business, while driving
speed and responsiveness as we serve consumers globally.
FINANCIAL HIGHLIGHTS
• In fiscal 2023, NIKE, Inc. achieved record Revenues of $51.2 billion, which increased 10% and 16% on a reported and currency-neutral basis, respectively
• NIKE Direct revenues grew 14% from $18.7 billion in fiscal 2022 to $21.3 billion in fiscal 2023, and represented approximately 44% of total NIKE Brand revenues for
fiscal 2023
• Gross margin for the fiscal year decreased 250 basis points to 43.5% primarily driven by higher product costs, higher markdowns and unfavorable changes in foreign
currency exchange rates, partially offset by strategic pricing action

In [18]:
results

[Document(id='48df4447-f488-451d-aba2-0d85a1d4757b', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../assets/example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 30, 'page_label': '31', 'start_index': 1540}, page_content='Enterprise Resource Planning Platform, data and analytics, demand sensing, insight gathering, and other areas to create an end-to-end technology foundation, which we\nbelieve will further accelerate our digital transformation. We believe this unified approach will accelerate growth and unlock more efficiency for our business, while driving\nspeed and responsiveness as we serve consumers glo

### 检索器（Retrievers）

LangChain 中的 `VectorStore` 对象 **没有继承** `Runnable` 接口。而 LangChain 的 **Retrievers（检索器）** 是 `Runnables`，因此它们实现了标准的一组方法（例如：同步和异步的 `invoke` 和 `batch` 操作）。

虽然我们可以通过向量存储来构建检索器，但检索器不仅可以与向量存储交互，还可以连接其他类型的数据源（例如外部 API）。

我们可以自己创建一个简单的版本，**不需要继承 Retriever 类**，也能实现类似功能。只要我们明确选择使用哪种方法来检索文档，就可以轻松地创建一个可运行的对象（Runnable）。下面我们将基于 `similarity_search` 方法来构建这样一个对象：

In [23]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='d1325b72-3b04-4371-a846-44cd80e5f06e', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../assets/example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are o

Vectorstores（向量存储）实现了一个名为 `as_retriever` 的方法，该方法会生成一个 Retriever（检索器），具体来说是一个 `VectorStoreRetriever`（向量存储检索器）。这些检索器包含特定的 `search_type` 和 `search_kwargs` 属性，用于指定调用底层向量存储的哪些方法，以及如何对它们进行参数化。例如，我们可以用以下方式复制上面的效果：

In [24]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='d1325b72-3b04-4371-a846-44cd80e5f06e', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../assets/example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are o

`VectorStoreRetriever` 支持三种搜索类型：  
- `"similarity"`（相似性），这是默认的搜索类型，  
- `"mmr"`（最大边缘相关性，用于在结果中平衡与查询的相关性和结果之间的多样性），  
- 以及 `"similarity_score_threshold"`（相似性得分阈值）。  

我们可以使用最后一种搜索类型（`"similarity_score_threshold"`）来根据文档的相似性得分设定一个阈值，只有得分超过该阈值的文档才会被检索器输出。

检索器可以轻松地集成到更复杂的应用中，例如**检索增强生成**（RAG）应用。  
这种应用会将用户提出的问题与从向量库中检索到的相关上下文结合起来，形成一个完整的提示词（prompt），然后输入给大型语言模型（LLM）进行回答。